In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
indiabetes = pd.read_csv("../input/pyms-diabete/diabete.csv")

In [ ]:
indiabetes.columns

A priori, on n'a pas de colonne inutile à la prédiction. On vérifie maintenant les données.

In [ ]:
indiabetes.count()

Les données sont renseignées (pas de NaN), on vérifie maintenant si on n'a pas de "0" représentant une absence de données. En consultant les informations du dataset, il y en a un peu pour le taux de glucose, la tension et l'IMC, et beaucoup pour l'épaisseur. On va donc supprimer les cas où le taux de glucose, la tension ou l'IMC valent 0, et, puisque l'épaisseur semble suivre une loi à peu près normale, on va lui donner la valeur moyenne.

On a aussi beaucoup de cas où le taux d'insuline vaut 0, ce qui est également (semblablement) une absence de mesure. On va devoir, dans ce cas, estimer le taux d'insuline en fonction d'autres paramètres.

In [ ]:
indiabetes=indiabetes[(indiabetes.glucose!=0) & (indiabetes.tension !=0) & (indiabetes.bmi!=0)]
indiabetes.count()

In [ ]:
plt.hist(indiabetes.thickness, bins=100)

In [ ]:
indiabetes.thickness=indiabetes.thickness.replace(0,np.nan)

In [ ]:
indiabetes=indiabetes.fillna(method="pad")

In [ ]:
plt.hist(indiabetes.thickness, bins=100)

In [ ]:
plt.hist(indiabetes.insulin, 100)

In [ ]:
indiabetes_where_insulin=indiabetes[indiabetes.insulin!=0]
plt.hist(indiabetes_where_insulin.insulin, 100)

In [ ]:

g = sns.lmplot(x="glucose", y="insulin", hue="diabete", data=indiabetes_where_insulin,
               palette="Set1")

In [ ]:
sns.regplot(x="glucose", y="insulin", data=indiabetes_where_insulin,
                 scatter_kws={"s": 80}, ci=None, order=5)

Il semble y avoir corrélation entre le taux d'insuline et le taux de glucose (ce qui n'est pas surpenant en soit). Le fait d'être diabétique ou non, en revanche, ne semble pas influer cette corrélation. On va donc supposer le taux d'insuline en fonction du taux de glucose.

In [ ]:
indiabetes.insulin=indiabetes.insulin.replace(0,np.nan)
indiabetes.insulin=indiabetes['insulin'].interpolate(method='polynomial', order=5)
indiabetes=indiabetes.dropna()
indiabetes.count()

In [ ]:
sns.regplot(x="glucose", y="insulin", data=indiabetes,
                 scatter_kws={"s": 80}, ci=None, order=5)

In [ ]:
plt.hist(indiabetes.insulin, 100)

Nous avons maintenant des données plus satisfaisantes. On va appliquer la méthodes des Random Forests à ce dataset.

In [ ]:
X = indiabetes.drop(['diabete'], axis=1)
y = indiabetes.diabete
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
print(confusion_matrix(y_test,y_rf))

Nous avons beaucoup de faux positifs (Et même, autant de faux positifs que de vrais négatifs !). On va essayer d'arranger ça en modifiant les hyperparamètres de la random forest.

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

In [ ]:
from sklearn import model_selection
param_grid = {
              'n_estimators': [10, 50,250, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)
rf_gs.fit(X_train, y_train)
print(rf_gs.best_params_)

In [ ]:
rf2=rf_gs.best_estimator_
y_rf2=rf2.predict(X_test)
print(classification_report(y_test, y_rf2))

In [ ]:
print(confusion_matrix(y_test,y_rf2))

On n'a eu aucune amélioration. On va essayer avec XGboost.

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))

In [ ]:
print(confusion_matrix(y_test,y_xgb))

Toujours pas d'amélioration.

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

In [ ]:
sns.pairplot(indiabetes, hue="diabete")

Les colonnes inutiles semblent être :
- tension
- thickness
- insulin
- pedigree

On va les drop et réessayer

In [ ]:
indiabetes_2=indiabetes.drop(["tension","thickness","insulin","pedigree"],axis=1)
sns.pairplot(indiabetes_2, hue="diabete")

In [ ]:
X_2 = indiabetes_2.drop(['diabete'], axis=1)
y_2 = indiabetes_2.diabete
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=1)

xgb_2  = XGB.XGBClassifier()
xgb_2.fit(X_train_2, y_train_2)
y_xgb_2 = xgb_2.predict(X_test_2)
print(confusion_matrix(y_test_2, y_xgb_2))
print(classification_report(y_test_2, y_xgb_2))

C'est encore pire qu'avant